In [1]:
import pandas as pd
import time

In [2]:
all_features = ["tweet_type", "Language", "tweet_timestamp", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01")
train['like_bool'] = train.like_timestamp.fillna(0)
train.loc[train.like_bool != 0.0, 'like_bool'] = 1.0
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()
train = train[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]
train_set = train_set[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

val_set = val_set[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

In [3]:
def bool_to_str(value):
    "value should be a bool"
    return 'Yes' if value else 'No'

In [4]:
def construct_line(row, columns, dtypes, string_dict, file_obj, index, time1):
    
    if index % 100000 == 0:
        print(str(time.time() - time1) +": "+ str(index))
        
    global categorical_index
    numerical_index = 0
    new_line = str(row[-1])+ " "
    for i in range(len(columns) - 1):
        value_at_i = row[i]
        dtype_at_i = dtypes[i]
        column_name_at_i = columns[i]
        
        if ((dtype_at_i == object) or (dtype_at_i == bool)): # Categorical Features
            if (dtype_at_i == bool):
                value_at_i =  bool_to_str(value_at_i)
            
            value_to_be_found = column_name_at_i +"_"+ value_at_i
            
            if value_to_be_found in string_dict:
                indexed_value = string_dict[value_to_be_found]
                new_line = new_line + str(indexed_value)+":1 "
            
            else:
                indexed_value = categorical_index
                categorical_index = categorical_index + 1
                new_line = new_line + str(indexed_value)+":1 "
                string_dict[value_to_be_found] = indexed_value
                
        else: # Numerical Features
            new_line = new_line + str(numerical_index) + ":" +str(value_at_i)+" "
            numerical_index = numerical_index + 1
    file_obj.write(new_line+"\n")

In [ ]:
cnames_numerical = list(train_set.select_dtypes(exclude=['object', 'bool']).columns)
categorical_index = len(cnames_numerical) - 1 # Categorical indices start from here.
string_dict = {}
f_train_set = open("f1/like/train_set.csv", 'a')
print(len(train_set))
time1 = time.time()
train_set.apply(lambda x : construct_line(x, train_set.columns, train_set.dtypes, string_dict, f_train_set, x.name,\
                                         time1), axis = 1)
f_train_set.close()
print(len(val_set))
f_val_set = open("f1/like/val_set.csv", 'a')
val_set.apply(lambda x : construct_line(x, val_set.columns, val_set.dtypes, string_dict, f_val_set, x.name,\
                                       time1), axis = 1)
f_val_set.close()

133267714
138.7078287601471: 0
175.3887493610382: 100000
212.16091299057007: 200000
248.91945600509644: 300000
285.6941740512848: 400000
322.27205634117126: 500000
359.17601680755615: 600000
398.41511607170105: 700000
437.5908191204071: 800000
476.8905680179596: 900000
516.1540327072144: 1000000
555.3687949180603: 1100000
594.6332190036774: 1200000
633.8358178138733: 1300000
672.9646909236908: 1400000
712.2008121013641: 1500000
751.4459059238434: 1600000
790.5723648071289: 1700000
829.848655462265: 1800000
868.9678375720978: 1900000
908.1261959075928: 2000000
947.4957690238953: 2100000
986.7196950912476: 2200000
1025.8938026428223: 2300000
1065.3887691497803: 2400000
1104.6075987815857: 2500000
1143.8548982143402: 2600000
1183.1851937770844: 2700000
1222.4670634269714: 2800000
1261.5852363109589: 2900000
1300.9414563179016: 3000000
1340.1344282627106: 3100000
1379.3684380054474: 3200000
1418.7381448745728: 3300000
1457.887405872345: 3400000
1497.051706790924: 3500000
1536.43097615242: 

11738.80192399025: 29500000
11776.367462396622: 29600000
11813.638655662537: 29700000
11850.976409435272: 29800000
11888.23908662796: 29900000
11927.068970441818: 30000000
11966.76809668541: 30100000
12006.39464879036: 30200000
12046.163062095642: 30300000
12085.783887147903: 30400000
12125.70556139946: 30500000
12165.390417814255: 30600000
12205.000833034515: 30700000
12244.83605337143: 30800000
12286.52967453003: 30900000
12326.336292028427: 31000000
12364.186054706573: 31100000
12401.321892023087: 31200000
12438.480995893478: 31300000
12475.797283887863: 31400000
12513.019318342209: 31500000
12550.30124783516: 31600000
12587.62012887001: 31700000
12624.965539216995: 31800000
12662.36979842186: 31900000
12699.700973510742: 32000000
12736.92810177803: 32100000
12774.394759178162: 32200000
12811.753713130951: 32300000
12849.166670084: 32400000
12886.744938850403: 32500000
12924.149194002151: 32600000
12961.608448505402: 32700000
12999.131291389465: 32800000
13036.511896848679: 32900000

22449.40626001358: 58100000
22486.534880399704: 58200000
22523.6664249897: 58300000
22560.882950782776: 58400000
22598.212789058685: 58500000
22635.573313236237: 58600000
22673.15265250206: 58700000
22710.645577430725: 58800000
22748.56229710579: 58900000
22785.820172786713: 59000000
22822.959735155106: 59100000
22860.174614667892: 59200000
22897.501731157303: 59300000
22934.69964981079: 59400000
22972.19654726982: 59500000
23009.708970546722: 59600000
23047.254429340363: 59700000
23084.61404132843: 59800000
23122.34246611595: 59900000
23159.8041536808: 60000000
23197.346267938614: 60100000
23234.783143758774: 60200000
23272.111545801163: 60300000
23309.748455524445: 60400000
23346.903160572052: 60500000
23384.45183777809: 60600000
23421.80413556099: 60700000
23459.249616622925: 60800000
23496.57451725006: 60900000
23533.968632936478: 61000000
23571.283592939377: 61100000
23608.617354154587: 61200000
23645.871799230576: 61300000
23683.148854732513: 61400000
23720.543405771255: 61500000

33113.058374643326: 86800000
33150.10694694519: 86900000
33187.29054093361: 87000000
33224.41072130203: 87100000
33261.57756948471: 87200000
33298.69906306267: 87300000
33335.73181438446: 87400000
33372.93423867226: 87500000
33409.98528337479: 87600000
33447.055770635605: 87700000
33484.17792510986: 87800000
33521.31380224228: 87900000
33558.424172878265: 88000000
33595.62178206444: 88100000
33632.747821092606: 88200000
33669.830286979675: 88300000
33707.03267788887: 88400000
33744.07689523697: 88500000
33781.15164542198: 88600000
33818.35766148567: 88700000
33855.408903598785: 88800000
33892.476971149445: 88900000
33929.598462581635: 89000000
33966.68813371658: 89100000
34003.77683711052: 89200000
34040.84874248505: 89300000
34078.05374264717: 89400000
34115.14324069023: 89500000
34152.3543241024: 89600000
34189.47676706314: 89700000
34226.66901874542: 89800000
34263.88955998421: 89900000
34301.07528877258: 90000000
34338.18105530739: 90100000
34375.278240680695: 90200000
34412.372597

In [6]:
len(val_set)

14807523

In [ ]:
string_dict

### Read Test Set

In [ ]:
all_features = ["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_timestamp"]

test = pd.read_csv("s3://recsys-challenge-2020/val_26_04_2020.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01")
test['like_bool'] = test.like_timestamp.fillna(0)
test.loc[test.like_bool != 0.0, 'like_bool'] = 1.0
test = test[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

### Write Train and Test Simultaneously

In [ ]:
cnames_numerical = list(train.select_dtypes(exclude=['object', 'bool']).columns)
categorical_index = len(cnames_numerical) - 1 # Categorical indices start from here.
string_dict = {}
f_train = open("f1/like/train.csv.libfm", 'a')
print(len(train))
time1 = time.time()
train.apply(lambda x : construct_line(x, train.columns, train.dtypes, string_dict, f_train, x.name,\
                                         time1), axis = 1)
f_train.close()
print(len(test))
f_test = open("f1/like/test.csv.libfm", 'a')
test.apply(lambda x : construct_line(x, test.columns, test.dtypes, string_dict, f_test, x.name,\
                                       time1), axis = 1)
f_test.close()

In [ ]:
string_dict

### Print Statements

In [ ]:
train_set.dtypes

In [ ]:
val_set.dtypes

In [ ]:
train_set.head()

In [ ]:
val_set.head()

In [ ]:
def construct_line_beta(row, columns, dtypes, string_dict, categorical_index):
    numerical_index = 0
    new_line = str(row[-1])+ " "
    for i in range(len(columns) - 1):
        if ((dtypes[i] == object) or (dtypes[i] == bool)):
            if (dtypes[i] == bool):
                row[i] =  bool_to_str(row[i])
            values = string_dict[columns[i]]
            if row[i] in [x[0] for x in values]:
                index = [x[0] for x in values].index(row[i])
                value_index = values[index][1]
                new_line = new_line + str(value_index)+":1 "
            else:
                value_index = sum(len(v) for v in string_dict.values()) + categorical_index
                new_line = new_line + str(value_index)+":1 "
                values.append((row[i], value_index))
                string_dict[columns[i]] = values
        else:
            new_line = new_line + str(numerical_index) + ":" +str(row[i])+" "
            numerical_index = numerical_index + 1
    print(new_line)

### For Stackoverflow